In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

In [2]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['test.csv',
 'structures',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [4]:
file_folder =  '../../data/feature'

In [5]:
mytrial = []

In [6]:
# mytrial = []
# #  tune hypterparameters
# def objective(trial):
        
#     max_depth = trial.suggest_int('max_depth', 2, 10)
#     max_bin = trial.suggest_int('max_bin', 10, 100)
#     eta = trial.suggest_uniform('eta', 0.01, 0.4)
#     colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.6, 1)
#     min_child_weight = trial.suggest_int('min_child_weight', 200, 600)
# #     n_estimators = trial.suggest_int('n_estimators', 100, 500)
#     subsample = trial.suggest_uniform('subsample', 0.6, 1.0)
#     reg_lambda = trial.suggest_loguniform('reg_lambda', 0.000001, 1.0)
#     reg_alpha = trial.suggest_loguniform('reg_alpha', 0.000001, 1.0)
        
#     args= param.copy()
#     args['model'] = {
#         'cls':'xgb.XGBRegressor',
#         'init':{
#             "max_depth":max_depth,
#             "max_bin":max_bin,
#             "eta":eta,
#             "colsample_bytree":colsample_bytree,
#             "min_child_weight":min_child_weight,
# #             "n_estimators":n_estimators,
#             "subsample":subsample,
#             "reg_lambda":reg_lambda,
#             "reg_alpha":reg_alpha,
#             'n_jobs':-1},
#         'fit':{},
#     }
    
#     df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train, args, f'tune hyper', None, trial=mytrial, is_output_feature_importance=False, trial_level=1)
    
#     val_mae = mean_absolute_error(df_valid_pred_i.predict.values, df_train.y.values)

#     return val_mae

# study = optuna.create_study()
# study.optimize(objective, n_trials=200)

In [7]:
# , 'fc', 'sd','pso','dso'
for y in ['scalar_coupling_constant']:
    for t in np.arange(0,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls':'CatBoostRegressor',
                'init':{
        #             "num_trees":num_trees,
        #             "depth":depth,
        #             "learning_rate":learning_rate,
        #             "l2_leaf_reg":l2_leaf_reg,
        #             "bagging_temperature":bagging_temperature,
        #             "random_strength":random_strength,
        #             "random_state":random_state,
                    'logging_level': 'Silent'
                },
                'fit':{
                },
            },
            'metric': 'mean_absolute_error'
        }

        df_his, df_feature_importances, df_valid_pred, df_test_pred = sk_process(df_train, param.copy(), 'no message', df_test=None, trial=mytrial, is_output_feature_importance=True, trial_level=1)
        
#         process_queue = PQueue(df_train, df_test, param, mytrial)
#         sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
#         select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
#         remove_useless = RFERemoveUselessFeaturesProcess(**{})
#         process_queue.insert_node(sort_features)
# #         process_queue.insert_node(select_topn)
# #         process_queue.insert_node(remove_useless)

#         try:
#             result = process_queue.run()
#         except Exception as e:
#             print(e.__str__())
#         print(len(process_queue.trial))
#         print(process_queue.param)
    

y=scalar_coupling_constant, type=0
y=scalar_coupling_constant, type=1
y=scalar_coupling_constant, type=2
y=scalar_coupling_constant, type=3
y=scalar_coupling_constant, type=4
y=scalar_coupling_constant, type=5
y=scalar_coupling_constant, type=6
y=scalar_coupling_constant, type=7


In [9]:
df_trial = pd.DataFrame(mytrial)

In [10]:
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-08-02 12:46:00.512292,no message,529,1.462795,1.475958,0.013163,no message
1,2019-08-02 14:11:41.075591,no message,529,0.928813,0.944770,0.015956,no message
2,2019-08-03 04:39:21.198456,no message,529,0.676458,0.682242,0.005784,no message
3,2019-08-03 15:36:29.174646,no message,529,0.318133,0.322310,0.004177,no message
4,2019-08-03 19:04:41.844539,no message,529,0.348252,0.359395,0.011142,no message
5,2019-08-04 09:33:53.734004,no message,529,0.750630,0.757314,0.006684,no message
6,2019-08-05 00:07:35.200229,no message,529,0.384765,0.387838,0.003073,no message
7,2019-08-05 05:05:25.619609,no message,529,0.243309,0.249623,0.006314,no message


In [16]:
df_trial['df_feature_importances'].values[0].head()

,feature,model_weight,weight,std,model_weight1,weight1,std1,model_weight2,weight2,std2,model_weight3,weight3,std3,model_weight4,weight4,std4
0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.010858,-5.177452e-07,8.224936e-07,0.0,0.0,0.0,0.0,0.0,0.0
1,E0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
2,E1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
3,EN_x,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
4,EN_y,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
_ = sort_feature_importances(df_trial['df_feature_importances'].values[0], 'average_permutation_weight')

In [20]:
df_trial['df_feature_importances'].values[0].sort_values(by=['average_model_weight'], ascending=False).head(50)['feature'].tolist()

['tertiary_angle_0',
 'atom_1_n_bonds',
 'distance',
 'atom_1_bond_lengths_mean',
 'inv_distPE',
 'inv_distP',
 'inv_distPR',
 'atom_1_bond_lengths_min',
 'inv_dist0R',
 'distance_c0',
 'dist_to_type_0_mean',
 'dist_to_type_1_mean',
 'dist_C_0_y',
 'distance_closest_1',
 'dist_to_type_mean',
 'linkM1',
 'atom_0_bond_lengths_mean',
 'atom_0_bond_lengths_max',
 'dist',
 'atom_0_bond_lengths_min',
 'distance_closest_0',
 'edge_4',
 'inv_dist1R',
 'yukawa_H.y',
 'dist_H_0_y',
 'dist_to_type_std',
 'min_distance_x',
 'atom_1_bond_lengths_std',
 'atom_index_1_cycle_size_mean',
 'vander_H.x',
 'distance_c1',
 'min_molecule_atom_0_dist_xyz',
 'dist_O_0_y',
 'bond_distance',
 'vander_H.y',
 'dist_O_0_x',
 'molecule_atom_index_1_dist_min',
 'min_molecule_atom_1_dist_xyz',
 'dist_1_neg_1',
 'dist_C_0_x',
 'L2dist',
 'min_distance_y',
 'dist_H_2_y',
 'tertiary_distance_2',
 'molecule_atom_index_0_dist_min',
 'dist_xyz',
 'bond_atom',
 'atom_1_bond_lengths_max',
 'molecule_atom_index_0_dist_max_div

In [21]:
lgbmtop50 = ['tertiary_angle_0',
 'inv_dist1R',
 'dist_to_type_std',
 'atom_1_n_bonds',
 'dist_O_0_x',
 'atom_1_bond_lengths_mean',
 'yukawa_H.y',
 'dist_C_0_y',
 'dist_xyz',
 'inv_dist0R',
 'atom_index_1_cycle_size_mean',
 'dist_O_0_y',
 'inv_dist1E',
 'inv_distPR',
 'tertiary_distance_2',
 'atom_1_bond_lengths_std',
 'vander_H.x',
 'eem_0',
 'eem_1',
 'dist_C_2_y',
 'dist_C_0_x',
 'dist_H_2_x',
 'dist_to_type_mean',
 'dist_O_1_y',
 'dist_N_0_y',
 'dist_C_3_x',
 'dist_N_0_x',
 'dist_H_2_y',
 'dist_C_1_y',
 'cos_f0',
 'dist_C_2_x',
 'dist_H_1_x',
 'dist_O_1_x',
 'tertiary_angle_3',
 'cos_center1',
 'atom_1_bond_lengths_max',
 'tertiary_angle_1',
 'dist_H_1_y',
 'dist_C_3_y',
 'dist_to_type_0_mean',
 'adC2',
 'molecule_atom_index_0_dist_max_div',
 'dist_C_4_y',
 'molecule_atom_index_1_dist_min_diff',
 'tertiary_distance_1',
 'tertiary_distance_4',
 'tertiary_angle_2',
 'dist_to_type_1_mean',
 'dist_H_0_y',
 'molecule_type_dist_mean_diff']

In [22]:
cattop50 = ['tertiary_angle_0',
 'atom_1_n_bonds',
 'distance',
 'atom_1_bond_lengths_mean',
 'inv_distPE',
 'inv_distP',
 'inv_distPR',
 'atom_1_bond_lengths_min',
 'inv_dist0R',
 'distance_c0',
 'dist_to_type_0_mean',
 'dist_to_type_1_mean',
 'dist_C_0_y',
 'distance_closest_1',
 'dist_to_type_mean',
 'linkM1',
 'atom_0_bond_lengths_mean',
 'atom_0_bond_lengths_max',
 'dist',
 'atom_0_bond_lengths_min',
 'distance_closest_0',
 'edge_4',
 'inv_dist1R',
 'yukawa_H.y',
 'dist_H_0_y',
 'dist_to_type_std',
 'min_distance_x',
 'atom_1_bond_lengths_std',
 'atom_index_1_cycle_size_mean',
 'vander_H.x',
 'distance_c1',
 'min_molecule_atom_0_dist_xyz',
 'dist_O_0_y',
 'bond_distance',
 'vander_H.y',
 'dist_O_0_x',
 'molecule_atom_index_1_dist_min',
 'min_molecule_atom_1_dist_xyz',
 'dist_1_neg_1',
 'dist_C_0_x',
 'L2dist',
 'min_distance_y',
 'dist_H_2_y',
 'tertiary_distance_2',
 'molecule_atom_index_0_dist_min',
 'dist_xyz',
 'bond_atom',
 'atom_1_bond_lengths_max',
 'molecule_atom_index_0_dist_max_div',
 'inv_dist1E']

In [25]:
set(lgbmtop50[:20]) - set(cattop50[:20])

{'atom_1_bond_lengths_std',
 'atom_index_1_cycle_size_mean',
 'dist_C_2_y',
 'dist_O_0_x',
 'dist_O_0_y',
 'dist_to_type_std',
 'dist_xyz',
 'eem_0',
 'eem_1',
 'inv_dist1E',
 'inv_dist1R',
 'tertiary_distance_2',
 'vander_H.x',
 'yukawa_H.y'}

In [9]:
df_train=pd.read_pickle(f'{file_folder}/df_train2.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test2.gzde', compression='gzip')
df_train['y'] = df_train['scalar_coupling_constant']

In [14]:
columns_list = [
{'columns': ['dist_C_3_y', 'eem_0', 'tertiary_angle_1', 'dist_C_4_x', 'dist_O_1_x', 'dist_C_4_y', 'inv_dist1', 'dist_H_0_x', 'inv_dist0R', 'min_molecule_atom_1_dist_xyz', 'cos_center1', 'dist_N_0_x', 'dist_C_2_y', 'dist_C_3_x', 'inv_dist1R', 'tertiary_atom_1', 'dist_O_0_y', 'vander_H.x', 'dist_H_3_y', 'tertiary_distance_0', 'max_molecule_atom_1_dist_xyz', 'dist_O_0_x', 'inv_distPR', 'tertiary_angle_3', 'tertiary_angle_2', 'dist_O_1_y', 'tertiary_distance_1', 'dist_C_1_y', 'tertiary_angle_4', 'yukawa_H.y', 'tertiary_distance_2', 'adC2', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'dist_H_1_y', 'molecule_atom_index_0_dist_max_diff', 'dist_N_0_y', 'tertiary_distance_5', 'dist_H_0_y', 'dist_to_type_std', 'atom_index_1_cycle_size_mean', 'adN1', 'dist_1_pos_1', 'dist_to_type_mean', 'dist_C_0_x', 'adC3', 'eem_1', 'atom_1_bond_lengths_mean', 'dist_H_2_x', 'molecule_atom_index_1_dist_min_diff', 'atom_1_n_bonds', 'cos_f0', 'tertiary_distance_3', 'dist_C_2_x', 'tertiary_angle_0', 'tertiary_atom_0', 'atom_1_bond_lengths_max', 'dist_H_2_y', 'dist_H_3_x', 'dist_to_type_0_mean', 'dist_1_neg_1', 'tertiary_distance_4', 'dist_to_type_1_mean', 'dist_C_0_y', 'yukawa_H.x', 'distC1', 'molecule_atom_index_0_dist_max_div'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_H_0_y', 'min_molecule_atom_1_dist_xyz', 'inv_dist1R', 'dist_C_0_y', 'dist_C_1_y', 'inv_distP', 'atom_1_bond_lengths_mean', 'bond_atom', 'tertiary_angle_1', 'dist_H_0_x', 'atom_1_bond_lengths_std', 'dist_H_1_y', 'yukawa_H.y', 'eem_0', 'tertiary_angle_0', 'dist_C_2_x', 'tertiary_distance_0', 'vander_N.x', 'dist_O_0_y', 'dist_O_0_x', 'dist_1_neg_1', 'tertiary_atom_0', 'inv_dist1', 'dist_N_0_y', 'dist_N_1_x', 'vander_H.x', 'atom_index_1_cycle_size_mean', 'dist_N_0_x', 'dist_C_1_x', 'inv_distPR', 'dist_C_3_x', 'dist_H_1_x', 'adC3', 'dist_C_2_y', 'tertiary_distance_3', 'tertiary_angle_2', 'inv_dist0R', 'dist_to_type_0_mean', 'cos_f0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'cos_center1', 'vander_N.y', 'dist_C_0_x', 'tertiary_distance_2', 'atom_1_bond_lengths_max', 'linkN', 'tertiary_distance_5', 'dist_C_3_y', 'molecule_atom_index_1_dist_max_div'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_H_0_y', 'min_molecule_atom_1_dist_xyz', 'inv_dist1R', 'dist_C_0_y', 'dist_C_1_y', 'inv_distP', 'atom_1_bond_lengths_mean', 'bond_atom', 'tertiary_angle_1', 'dist_H_0_x', 'atom_1_bond_lengths_std', 'dist_H_1_y', 'yukawa_H.y', 'eem_0', 'tertiary_angle_0', 'dist_C_2_x', 'tertiary_distance_0', 'vander_N.x', 'dist_O_0_y', 'dist_O_0_x', 'dist_1_neg_1', 'tertiary_atom_0', 'inv_dist1', 'dist_N_0_y', 'dist_N_1_x', 'vander_H.x', 'atom_index_1_cycle_size_mean', 'dist_N_0_x', 'dist_C_1_x', 'inv_distPR', 'dist_C_3_x', 'dist_H_1_x', 'adC3', 'dist_C_2_y', 'tertiary_distance_3', 'tertiary_angle_2', 'inv_dist0R', 'dist_to_type_0_mean', 'cos_f0', 'tertiary_distance_1', 'molecule_atom_index_0_dist_max_div', 'cos_center1', 'vander_N.y', 'dist_C_0_x', 'tertiary_distance_2', 'atom_1_bond_lengths_max', 'linkN', 'tertiary_distance_5', 'dist_C_3_y', 'molecule_atom_index_1_dist_max_div'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_N_0_y', 'adC1', 'dist_H_0_y', 'link0', 'tertiary_angle_1', 'inv_distP', 'dist_C_3_y', 'inv_distPE', 'dist_H_0_x', 'dist_to_type_std', 'cos_c1', 'inv_dist0R', 'adN1', 'dist_N_0_x', 'dist_C_2_y', 'dist_C_1_x', 'dist_to_type_mean', 'dist_C_3_x', 'cos_f1', 'dist_C_0_x', 'distance_c1', 'adC3', 'tertiary_atom_1', 'molecule_atom_index_1_dist_mean', 'cos_f0', 'tertiary_atom_2', 'dist_H_3_y', 'dist_C_2_x', 'tertiary_angle_0', 'max_distance_y', 'dist_O_0_x', 'coulomb_H.y', 'dist_H_2_y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'tertiary_angle_3', 'dist_H_3_x', 'tertiary_angle_2', 'dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_1', 'dist_C_1_y', 'yukawa_H.y', 'tertiary_distance_4', 'tertiary_distance_2', 'adC2', 'dist_to_type_1_mean', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_y', 'yukawa_H.x', 'molecule_dist_min', 'dist_H_1_x', 'cos_c0_c1', 'dist_H_1_y', 'cos_f0_f1', 'dist_O_0_y', 'mean_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_N_0_y', 'adC1', 'dist_H_0_y', 'link0', 'tertiary_angle_1', 'inv_distP', 'dist_C_3_y', 'inv_distPE', 'dist_H_0_x', 'dist_to_type_std', 'cos_c1', 'inv_dist0R', 'adN1', 'dist_N_0_x', 'dist_C_2_y', 'dist_C_1_x', 'dist_to_type_mean', 'dist_C_3_x', 'cos_f1', 'dist_C_0_x', 'distance_c1', 'adC3', 'tertiary_atom_1', 'molecule_atom_index_1_dist_mean', 'cos_f0', 'tertiary_atom_2', 'dist_H_3_y', 'dist_C_2_x', 'tertiary_angle_0', 'max_distance_y', 'dist_O_0_x', 'coulomb_H.y', 'dist_H_2_y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'tertiary_angle_3', 'dist_H_3_x', 'tertiary_angle_2', 'dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_1', 'dist_C_1_y', 'yukawa_H.y', 'tertiary_distance_4', 'tertiary_distance_2', 'adC2', 'dist_to_type_1_mean', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_y', 'yukawa_H.x', 'molecule_dist_min', 'dist_H_1_x', 'cos_c0_c1', 'dist_H_1_y', 'cos_f0_f1', 'dist_O_0_y', 'mean_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_N_0_y', 'adC1', 'dist_H_0_y', 'link0', 'tertiary_angle_1', 'inv_distP', 'dist_C_3_y', 'inv_distPE', 'dist_H_0_x', 'dist_to_type_std', 'cos_c1', 'inv_dist0R', 'adN1', 'dist_N_0_x', 'dist_C_2_y', 'dist_C_1_x', 'dist_to_type_mean', 'dist_C_3_x', 'cos_f1', 'dist_C_0_x', 'distance_c1', 'adC3', 'tertiary_atom_1', 'molecule_atom_index_1_dist_mean', 'cos_f0', 'tertiary_atom_2', 'dist_H_3_y', 'dist_C_2_x', 'tertiary_angle_0', 'max_distance_y', 'dist_O_0_x', 'coulomb_H.y', 'dist_H_2_y', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'tertiary_angle_3', 'dist_H_3_x', 'tertiary_angle_2', 'dist_xyz', 'dist_to_type_0_mean', 'tertiary_distance_1', 'dist_C_1_y', 'yukawa_H.y', 'tertiary_distance_4', 'tertiary_distance_2', 'adC2', 'dist_to_type_1_mean', 'molecule_atom_index_0_dist_min_div', 'dist_C_0_y', 'yukawa_H.x', 'molecule_dist_min', 'dist_H_1_x', 'cos_c0_c1', 'dist_H_1_y', 'cos_f0_f1', 'dist_O_0_y', 'mean_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_2_y', 'dist_to_type_std', 'tertiary_angle_3', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'dist_to_type_1_mean', 'inv_dist1E', 'adN1', 'dist_N_0_y', 'molecule_atom_index_1_dist_max_diff', 'dist_to_type_mean', 'tertiary_distance_2', 'max_molecule_atom_1_dist_xyz', 'tertiary_angle_2', 'molecule_atom_index_0_dist_max_diff', 'max_distance_y', 'tertiary_atom_0', 'dist_H_0_y', 'dist_H_2_y', 'dist_C_0_y', 'dist_C_1_y', 'max_molecule_atom_0_dist_xyz', 'cos_f0', 'adC3', 'distH0', 'dist_to_type_0_mean', 'dist_H_0_x', 'tertiary_distance_1', 'dist_H_1_y', 'tertiary_atom_2', 'dist_C_2_x', 'dihedral', 'tertiary_angle_1', 'tertiary_atom_1', 'molecule_atom_index_0_dist_min_div', 'cos_c1', 'distance_c1', 'dist_C_0_x', 'dist_O_0_x', 'tertiary_angle_0', 'dist_C_3_y', 'cos_f1', 'dist_O_0_y', 'tertiary_distance_3', 'dist_C_1_x', 'atom_index_closest_1', 'adC2', 'cos_c0_c1'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
{'columns': ['dist_C_3_y', 'adC1', 'tertiary_angle_1', 'dist_H_0_y', 'dist_to_type_std', 'atom_index_1_cycle_size_mean', 'cos_c1', 'molecule_atom_index_0_dist_std_diff', 'dist_N_0_x', 'dist_C_1_x', 'dist_C_2_y', 'dist_C_3_x', 'tertiary_atom_1', 'atom_1_bond_lengths_mean', 'atom_1_n_bonds', 'cos_f0', 'tertiary_distance_3', 'dist_C_2_x', 'tertiary_angle_0', 'atom_0_bond_lengths_max', 'dist_O_0_x', 'molecule_atom_index_0_dist_min_diff', 'cos_c0', 'tertiary_angle_3', 'tertiary_angle_2', 'tertiary_distance_1', 'dist_C_1_y', 'tertiary_distance_2', 'molecule_atom_index_0_dist_min_div', 'dist_to_type_1_mean', 'dist_C_0_y', 'max_molecule_atom_0_dist_xyz', 'dist_H_1_x', 'atom_1_bond_lengths_std', 'cos_c0_c1', 'dist_O_0_y', 'mean_molecule_atom_0_dist_xyz'], 'cv': {'cls': 'GroupKFold', 'init': {'n_splits': 5}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'},
]

In [15]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [1]:
# df_trial = pd.DataFrame(mytrial)
# df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

In [13]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.46002675694877715
0 0.9396508899993137
3 0.19993346218512523
1 0.43385297186382876
4 0.2349290397789158
2 0.44128424079965317
6 0.22103501825466695
5 0.4610286745311185
7 0.14194919234848902


In [18]:
idx=0.448
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)